In [5]:
import requests
from bs4 import BeautifulSoup
import time

regions = {}
current_id = 1
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

REQUEST_TIMEOUT = 10
DELAY_BETWEEN_REQUESTS = 1.5
LARGE_DELAY_BETWEEN_REQUESTS = 10
NOT_200_COUNTER = 0
EMPTY_NEXT_ANSWER_COUNTER = 0

In [11]:
EMPTY_NEXT_ANSWER_COUNTER = 0

In [12]:

while True:
    url = f"https://bolshayastrana.com/tury?regions={current_id}"
    
    try:
        response = requests.get(url, headers=headers, timeout=REQUEST_TIMEOUT)
        
        if response.status_code == 200:
            NOT_200_COUNTER = 0
            soup = BeautifulSoup(response.text, 'html.parser')
            region_input = soup.find('input', class_='search-form-field__input')
            
            if region_input:
                region_name = region_input.get('value', region_input.get('title', '')).strip()
                if region_name:
                    regions[region_name] = str(current_id)
                    if current_id % 10 == 0:
                        print(f"✅ Успех: {region_name} (ID: {current_id})")
                    current_id += 1
                    EMPTY_NEXT_ANSWER_COUNTER = 0
                    time.sleep(DELAY_BETWEEN_REQUESTS)
                    continue
            
            # Если дошли сюда - город пустой или input не найден при статусе 200
            if EMPTY_NEXT_ANSWER_COUNTER == 100:
                print(f"🛑 Пустой результат для ID {current_id}. Попытка номер {EMPTY_NEXT_ANSWER_COUNTER}. Завершаем сбор.")
                break

            EMPTY_NEXT_ANSWER_COUNTER += 1
            print(f"🛑 Пустой результат для ID {current_id}. Пробуем следующий.")
            current_id += 1
            time.sleep(DELAY_BETWEEN_REQUESTS)
            continue
            
        else:
            if NOT_200_COUNTER >= 3:
                print(f"🛑 На ID {current_id} так и не был получен 200 статус, пауза.")
                break

            print(f"⚠️ На ID {current_id} получена ошибка, повторная итерация {NOT_200_COUNTER} (статус {response.status_code})")
            NOT_200_COUNTER += 1
            time.sleep(LARGE_DELAY_BETWEEN_REQUESTS)

            
    except requests.exceptions.Timeout:
        print(f"⌛ Таймаут для ID {current_id}.")
    except requests.exceptions.ConnectionError:
        print(f"🔌 Ошибка соединения для ID {current_id}. Повторяем через 5 секунд...")
        time.sleep(5)
    except Exception as e:
        print(f"❌ Ошибка при обработке ID {current_id}: {str(e)}")
        time.sleep(DELAY_BETWEEN_REQUESTS)

🛑 Пустой результат для ID 216. Пробуем следующий.
🛑 Пустой результат для ID 217. Пробуем следующий.
🛑 Пустой результат для ID 218. Пробуем следующий.
🛑 Пустой результат для ID 219. Пробуем следующий.
🛑 Пустой результат для ID 220. Пробуем следующий.
🛑 Пустой результат для ID 221. Пробуем следующий.
🛑 Пустой результат для ID 222. Пробуем следующий.
🛑 Пустой результат для ID 223. Пробуем следующий.
🛑 Пустой результат для ID 224. Пробуем следующий.
🛑 Пустой результат для ID 225. Пробуем следующий.
🛑 Пустой результат для ID 226. Пробуем следующий.
🛑 Пустой результат для ID 227. Пробуем следующий.
🛑 Пустой результат для ID 228. Пробуем следующий.
🛑 Пустой результат для ID 229. Пробуем следующий.
🛑 Пустой результат для ID 230. Пробуем следующий.
🛑 Пустой результат для ID 231. Пробуем следующий.
🛑 Пустой результат для ID 232. Пробуем следующий.
🛑 Пустой результат для ID 233. Пробуем следующий.
🛑 Пустой результат для ID 234. Пробуем следующий.
🛑 Пустой результат для ID 235. Пробуем следующий.


In [ ]:
dict_regions = regions.copy()

#### Sample for debug

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://bolshayastrana.com/tury?regions=1"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    region_input = soup.find('input', class_='search-form-field__input')
    
    if region_input:
        region_name = region_name = region_input.get('value', region_input.get('title', '')).strip()
        print(f"Название города: {region_name}")
    else:
        print("Не удалось найти название города в HTML")
        
except requests.exceptions.RequestException as e:
    print(f"Ошибка при запросе к странице: {e}")
except Exception as e:
    print(f"Произошла ошибка: {e}")

Название города: Алтай


### Регионы РФ

In [16]:

import pandas as pd
import re

url = "https://ru.wikipedia.org/wiki/Субъекты_Российской_Федерации"

response = requests.get(url)
response.raise_for_status()  # Проверяем, что запрос успешен

soup = BeautifulSoup(response.text, 'html.parser')

# Находим таблицу с городами (может потребоваться уточнение)
table = soup.find('table', {'class': 'standard'})

if not table:
    print("Таблица не найдена!")

# Извлекаем заголовки столбцов
headers = [th.get_text(strip=True) for th in table.find('tr').find_all('th')]

# Собираем данные
data = []
for row in table.find_all('tr')[1:]:  # Пропускаем строку с заголовками
    columns = row.find_all(['td', 'th'])  # Обрабатываем и td, и th
    
    row_data = []
    for col in columns:
        text = col.get_text(' ', strip=True)
        text = text.replace('\xa0', ' ')  # Заменяем неразрывные пробелы
        row_data.append(text)
    
    if row_data:
        data.append(row_data)

# Создаем DataFrame
df = pd.DataFrame(data, columns=headers)

# Функция для очистки числовых значений
def clean_number(value):
    if pd.isna(value):
        return None
    # Удаляем всё, кроме цифр и знака минуса (для отрицательных чисел)
    cleaned = re.sub(r'[^\d-]', '', str(value))
    return int(cleaned) if cleaned else None

# Очищаем числовые столбцы
df['Население01.01.2024[17]'] = df['Население01.01.2024[17]'].apply(clean_number)


In [17]:
df

,№,Субъект Российской Федерации,Флаг,Герб,Терри-тория (км²),Население01.01.2024[17],Адм. центр/столица,Адм.-тер. устройство(согласноОКАТО),КодОКАТО,Муниципальные образования,История адм.-тер. деления
0,1e-06,Республики,,,,NaN,,,,,
1,1,Республика Адыгея,,,7792,500591.0,Майкоп,7 районов и 2 города,79,"7 муниципальных районов, 2 городских округа",подробнее
2,2,Республика Алтай,,,92903,210765.0,Горно-Алтайск,10 районов и 1 город,84,"10 муниципальных районов, 1 городской округ",подробнее
3,3,Республика Башкортостан,,,142947,4064361.0,Уфа,54 района и 21 город,80,"54 муниципальных района, 9 городских округов",подробнее
4,4,Республика Бурятия,,,351334,971922.0,Улан-Удэ,21 район и 2 города,81,"21 муниципальный район, 2 городских округа",подробнее
...,...,...,...,...,...,...,...,...,...,...,...
91,86,Ненецкий АО [ 11 ],,,176810,42224.0,Нарьян-Мар,1 район и 1 город,11,"1 муниципальный район, 1 городской округ",подробнее
92,87,Ханты-Мансийский АО — Югра [ 14 ],,,534801,1759386.0,Ханты-Мансийск,9 районов и 14 городов,71,"9 муниципальных районов, 13 городских округов",подробнее
93,88,Чукотский АО,,,721481,48029.0,Анадырь,8 районов и 1 город,77,"6 муниципальных районов, 1 городской округ",подробнее
94,89,Ямало-Ненецкий АО [ 14 ],,,769250,515960.0,Салехард,7 районов и 8 городов,71,"7 муниципальных районов, 6 городских округов",подробнее


In [ ]:
def normalize_name(name):
    name = str(name).lower()
    prefixes = ["республика", "город", "муниципальный округ", "край", "область"]
    for prefix in prefixes:
        if name.startswith(prefix):
            name = name.replace(prefix, "").strip()
    return name.strip()


df_normalized = {normalize_name(region): region for region in df["Субъект Российской Федерации"]} 
dict_normalized = {normalize_name(region): code for region, code in dict_regions.items()} 

filtered_dict = {
    region: code  # Оригинальное название из словаря
    for region, code in dict_regions.items()
    if normalize_name(region) in df_normalized
}

In [22]:
print(f"Было: {len(dict_regions)} регион")
print(f"Стало: {len(filtered_dict)} регион")
print("Пример отфильтрованного словаря:", dict(list(filtered_dict.items())[:5]))

Было: 150 регион
Стало: 51 регион
Пример отфильтрованного словаря: {'Алтай': '1', 'Адыгея': '3', 'Карелия': '14', 'Калмыкия': '19', 'Хакасия': '24'}


In [23]:
filtered_dict

{'Алтай': '1',
 'Адыгея': '3',
 'Карелия': '14',
 'Калмыкия': '19',
 'Хакасия': '24',
 'Дагестан': '26',
 'Ингушетия': '29',
 'Калужская область': '33',
 'Орловская область': '34',
 'Рязанская область': '35',
 'Белгородская область': '36',
 'Воронежская область': '37',
 'Тверская область': '38',
 'Псковская область': '39',
 'Ярославская область': '40',
 'Магаданская область': '41',
 'Ленинградская область': '42',
 'Тульская область': '43',
 'Костромская область': '44',
 'Липецкая область': '45',
 'Владимирская область': '46',
 'Калининградская область': '48',
 'Ивановская область': '49',
 'Хабаровский край': '52',
 'Нижегородская область': '53',
 'Красноярский край': '55',
 'Коми': '56',
 'Бурятия': '57',
 'Пермский край': '64',
 'Астраханская область': '68',
 'Ростовская область': '69',
 'Амурская область': '70',
 'Вологодская область': '74',
 'Московская область': '75',
 'Кировская область': '77',
 'Челябинская область': '83',
 'Мурманская область': '85',
 'Краснодарский край': '94',

In [25]:
import json

with open("dict_regions.json", "w", encoding="utf-8") as f:
    json.dump(filtered_dict, f, ensure_ascii=False, indent=4)

with open("dict_regions_FULL.json", "w", encoding="utf-8") as f:
    json.dump(dict_regions, f, ensure_ascii=False, indent=4)